# Out-of-State-Contributions: States Analysis

How much out-of-state money have candidates in each of the states raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [1]:
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

<IPython.core.display.Javascript object>

Import contributions data.

In [2]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/jyerardi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,17,18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6514274 entries, 0 to 6514273
Data columns (total 23 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           float64
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(2), int64(2), object(19)
memory usage: 1.1+ GB


<IPython.core.display.Javascript object>

Convert the contribution date and latest month columns to datetime data type.

In [3]:
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6514274 entries, 0 to 6514273
Data columns (total 23 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           float64
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(2), int64(2), object(17)
memory usage: 1.1+ GB


## Calculate out-of-state contributions by state, office and party

Filter the data by election cycle.

In [4]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_14 = contributions[contributions["year"] == 2014]
contributions_10 = contributions[contributions["year"] == 2010]

Group contributions by state, standardized office, party and in-vs.-out-of-state status and sum contributions.

In [5]:
contributions_by_state_18 = contributions_18.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_18.rename(columns={"amount": "contributions_18"}, inplace=True)
contributions_by_state_14 = contributions_14.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_14.rename(columns={"amount": "contributions_14"}, inplace=True)
contributions_by_state_10 = contributions_10.groupby(["state", "standardized_office", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_state_10.rename(columns={"amount": "contributions_10"}, inplace=True)
contributions_by_state_18.head()

,state,standardized_office,party,in_out_state,contributions_18
0,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,in-state,"188,649.92"
1,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,out-of-state,"23,657.00"
2,AK,GOVERNOR/LIEUTENANT GOVERNOR,Democratic,unknown,250.00
3,AK,GOVERNOR/LIEUTENANT GOVERNOR,Republican,in-state,"841,763.22"
4,AK,GOVERNOR/LIEUTENANT GOVERNOR,Republican,out-of-state,"22,129.00"


Pivot dataframe to aggregate each state and party's data in a single row.

In [6]:
contributions_by_state_18 = pd.pivot_table(contributions_by_state_18, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_14 = pd.pivot_table(contributions_by_state_14, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()
contributions_by_state_10 = pd.pivot_table(contributions_by_state_10, index=["state", "party"], columns=["standardized_office", "in_out_state"]).reset_index()

It appears that some states have no contributions categorized as "unknown." Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [7]:
contributions_by_state_18.fillna(0, inplace=True)
contributions_by_state_14.fillna(0, inplace=True)
contributions_by_state_10.fillna(0, inplace=True)

Flatten the resulting dataframes' multi-index columns.

In [8]:
contributions_by_state_18.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_18.columns.values]
contributions_by_state_14.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_14.columns.values]
contributions_by_state_10.columns = ["_".join(column).replace(" ","_").replace("-","_").strip("_").lower() for column in contributions_by_state_10.columns.values]

Calculate the total contributions by state and office.

In [9]:
contributions_by_state_14

,state,party,contributions_14_governor/lieutenant_governor_in_state,contributions_14_governor/lieutenant_governor_out_of_state,contributions_14_governor/lieutenant_governor_unknown,contributions_14_state_house/assembly/senate_in_state,contributions_14_state_house/assembly/senate_out_of_state,contributions_14_state_house/assembly/senate_unknown
0,AK,Democratic,"754,909.42","64,737.07",200.00,"998,192.22","48,870.67",0.00
1,AK,Republican,"898,969.23","49,058.00",0.00,"1,750,230.45","54,957.87",0.00
2,AK,Third-Party,"477,430.44","26,566.19",0.00,"75,489.74","3,168.32",0.00
3,AL,Democratic,"221,924.98","4,926.00",0.00,"4,830,536.35","374,601.14","5,602.53"
4,AL,Republican,"5,356,915.70","705,138.00","8,000.00","20,933,846.45","1,122,865.53","14,500.00"
5,AL,Third-Party,0.00,0.00,0.00,"243,108.14","4,800.00",0.00
6,AR,Democratic,"4,175,790.56","497,006.59","10,950.45","1,433,398.51","174,791.00","43,914.13"
7,AR,Republican,"2,333,827.25","407,517.76","6,100.00","2,239,116.09","313,988.25","44,597.38"
8,AR,Third-Party,"2,863.74",100.00,0.00,0.00,0.00,0.00
9,CA,Democratic,"19,679,376.74","2,836,473.92",0.00,"56,278,644.93","8,143,094.37","18,497.00"


In [10]:
# 2018
contributions_by_state_18["governor/lieutenant_governor_total"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"]
contributions_by_state_18["state_house/assembly/senate_total"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"]
# 2014
contributions_by_state_14["governor/lieutenant_governor_total"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"]
contributions_by_state_14["state_house/assembly/senate_total"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"]
# 2010
contributions_by_state_10["governor/lieutenant_governor_total"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"]
contributions_by_state_10["state_house/assembly/senate_total"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions.

In [11]:
contributions_by_state_18["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"] / (contributions_by_state_18["contributions_18_governor/lieutenant_governor_in_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_out_of_state"] + contributions_by_state_18["contributions_18_governor/lieutenant_governor_unknown"])
contributions_by_state_18["pct_state_house/assembly/senate_in_state"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] / (contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"])
contributions_by_state_18["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] / (contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"])
contributions_by_state_18["pct_state_house/assembly/senate_unknown"] = contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"] / (contributions_by_state_18["contributions_18_state_house/assembly/senate_in_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_out_of_state"] + contributions_by_state_18["contributions_18_state_house/assembly/senate_unknown"])

In [12]:
contributions_by_state_18

,state,party,contributions_18_governor/lieutenant_governor_in_state,contributions_18_governor/lieutenant_governor_out_of_state,contributions_18_governor/lieutenant_governor_unknown,contributions_18_state_house/assembly/senate_in_state,contributions_18_state_house/assembly/senate_out_of_state,contributions_18_state_house/assembly/senate_unknown,governor/lieutenant_governor_total,state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"188,649.92","23,657.00",250.00,"1,323,660.05","59,793.09","3,980.00","212,556.92","1,387,433.14",0.89,0.11,0.00,0.95,0.04,0.00
1,AK,Republican,"841,763.22","22,129.00",58.86,"1,516,941.73","66,269.75",5.00,"863,951.08","1,583,216.48",0.97,0.03,0.00,0.96,0.04,0.00
2,AK,Third-Party,"766,672.29","47,474.00",0.00,"6,612.34",50.00,0.00,"814,146.29","6,662.34",0.94,0.06,0.00,0.99,0.01,0.00
3,AL,Democratic,"1,912,675.57","78,671.07","-16,485.22","4,030,093.37","346,186.42","-30,108.28","1,974,861.42","4,346,171.51",0.97,0.04,-0.01,0.93,0.08,-0.01
4,AL,Republican,"12,984,811.72","896,446.07","738,622.84","16,950,989.34","1,123,204.60","-52,486.64","14,619,880.63","18,021,707.30",0.89,0.06,0.05,0.94,0.06,-0.00
5,AL,Third-Party,0.00,0.00,0.00,"40,500.00",0.00,0.00,0.00,"40,500.00",nan,nan,nan,1.00,0.00,0.00
6,AR,Democratic,"204,371.47","81,932.52",0.00,"842,398.53","183,400.67","27,553.01","286,303.99","1,053,352.21",0.71,0.29,0.00,0.80,0.17,0.03
7,AR,Republican,"3,786,607.43","399,807.00",0.00,"2,597,329.29","250,255.20","109,883.84","4,186,414.43","2,957,468.33",0.90,0.10,0.00,0.88,0.08,0.04
8,AR,Third-Party,375.00,0.00,0.00,"15,345.00",250.00,0.00,375.00,"15,595.00",1.00,0.00,0.00,0.98,0.02,0.00
9,CA,Democratic,"60,117,372.02","6,812,521.35","123,140.98","54,012,750.66","9,755,234.52","38,160.39","67,053,034.35","63,806,145.57",0.90,0.10,0.00,0.85,0.15,0.00


In [13]:
cols = contributions_by_state_18.columns.tolist()
cols = cols[0:5] + cols[8:9] + cols[5:8] + cols[9:10] + cols[10:16]
contributions_by_state_18 = contributions_by_state_18[cols]
contributions_by_state_18

,state,party,contributions_18_governor/lieutenant_governor_in_state,contributions_18_governor/lieutenant_governor_out_of_state,contributions_18_governor/lieutenant_governor_unknown,governor/lieutenant_governor_total,contributions_18_state_house/assembly/senate_in_state,contributions_18_state_house/assembly/senate_out_of_state,contributions_18_state_house/assembly/senate_unknown,state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"188,649.92","23,657.00",250.00,"212,556.92","1,323,660.05","59,793.09","3,980.00","1,387,433.14",0.89,0.11,0.00,0.95,0.04,0.00
1,AK,Republican,"841,763.22","22,129.00",58.86,"863,951.08","1,516,941.73","66,269.75",5.00,"1,583,216.48",0.97,0.03,0.00,0.96,0.04,0.00
2,AK,Third-Party,"766,672.29","47,474.00",0.00,"814,146.29","6,612.34",50.00,0.00,"6,662.34",0.94,0.06,0.00,0.99,0.01,0.00
3,AL,Democratic,"1,912,675.57","78,671.07","-16,485.22","1,974,861.42","4,030,093.37","346,186.42","-30,108.28","4,346,171.51",0.97,0.04,-0.01,0.93,0.08,-0.01
4,AL,Republican,"12,984,811.72","896,446.07","738,622.84","14,619,880.63","16,950,989.34","1,123,204.60","-52,486.64","18,021,707.30",0.89,0.06,0.05,0.94,0.06,-0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"40,500.00",0.00,0.00,"40,500.00",nan,nan,nan,1.00,0.00,0.00
6,AR,Democratic,"204,371.47","81,932.52",0.00,"286,303.99","842,398.53","183,400.67","27,553.01","1,053,352.21",0.71,0.29,0.00,0.80,0.17,0.03
7,AR,Republican,"3,786,607.43","399,807.00",0.00,"4,186,414.43","2,597,329.29","250,255.20","109,883.84","2,957,468.33",0.90,0.10,0.00,0.88,0.08,0.04
8,AR,Third-Party,375.00,0.00,0.00,375.00,"15,345.00",250.00,0.00,"15,595.00",1.00,0.00,0.00,0.98,0.02,0.00
9,CA,Democratic,"60,117,372.02","6,812,521.35","123,140.98","67,053,034.35","54,012,750.66","9,755,234.52","38,160.39","63,806,145.57",0.90,0.10,0.00,0.85,0.15,0.00


In [14]:
contributions_by_state_14["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"] / (contributions_by_state_14["contributions_14_governor/lieutenant_governor_in_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_out_of_state"] + contributions_by_state_14["contributions_14_governor/lieutenant_governor_unknown"])
contributions_by_state_14["pct_state_house/assembly/senate_in_state"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] / (contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"])
contributions_by_state_14["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] / (contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"])
contributions_by_state_14["pct_state_house/assembly/senate_unknown"] = contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"] / (contributions_by_state_14["contributions_14_state_house/assembly/senate_in_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_out_of_state"] + contributions_by_state_14["contributions_14_state_house/assembly/senate_unknown"])

In [15]:
cols = contributions_by_state_14.columns.tolist()
cols = cols[0:5] + cols[8:9] + cols[5:8] + cols[9:10] + cols[10:16]
contributions_by_state_14 = contributions_by_state_14[cols]
contributions_by_state_14

,state,party,contributions_14_governor/lieutenant_governor_in_state,contributions_14_governor/lieutenant_governor_out_of_state,contributions_14_governor/lieutenant_governor_unknown,governor/lieutenant_governor_total,contributions_14_state_house/assembly/senate_in_state,contributions_14_state_house/assembly/senate_out_of_state,contributions_14_state_house/assembly/senate_unknown,state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"754,909.42","64,737.07",200.00,"819,846.49","998,192.22","48,870.67",0.00,"1,047,062.89",0.92,0.08,0.00,0.95,0.05,0.00
1,AK,Republican,"898,969.23","49,058.00",0.00,"948,027.23","1,750,230.45","54,957.87",0.00,"1,805,188.32",0.95,0.05,0.00,0.97,0.03,0.00
2,AK,Third-Party,"477,430.44","26,566.19",0.00,"503,996.63","75,489.74","3,168.32",0.00,"78,658.06",0.95,0.05,0.00,0.96,0.04,0.00
3,AL,Democratic,"221,924.98","4,926.00",0.00,"226,850.98","4,830,536.35","374,601.14","5,602.53","5,210,740.02",0.98,0.02,0.00,0.93,0.07,0.00
4,AL,Republican,"5,356,915.70","705,138.00","8,000.00","6,070,053.70","20,933,846.45","1,122,865.53","14,500.00","22,071,211.98",0.88,0.12,0.00,0.95,0.05,0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"243,108.14","4,800.00",0.00,"247,908.14",nan,nan,nan,0.98,0.02,0.00
6,AR,Democratic,"4,175,790.56","497,006.59","10,950.45","4,683,747.60","1,433,398.51","174,791.00","43,914.13","1,652,103.64",0.89,0.11,0.00,0.87,0.11,0.03
7,AR,Republican,"2,333,827.25","407,517.76","6,100.00","2,747,445.01","2,239,116.09","313,988.25","44,597.38","2,597,701.72",0.85,0.15,0.00,0.86,0.12,0.02
8,AR,Third-Party,"2,863.74",100.00,0.00,"2,963.74",0.00,0.00,0.00,0.00,0.97,0.03,0.00,nan,nan,nan
9,CA,Democratic,"19,679,376.74","2,836,473.92",0.00,"22,515,850.66","56,278,644.93","8,143,094.37","18,497.00","64,440,236.30",0.87,0.13,0.00,0.87,0.13,0.00


In [16]:
contributions_by_state_10["pct_governor/lieutenant_governor_in_state"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_governor/lieutenant_governor_out_of_state"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_governor/lieutenant_governor_unknown"] = contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"] / (contributions_by_state_10["contributions_10_governor/lieutenant_governor_in_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_out_of_state"] + contributions_by_state_10["contributions_10_governor/lieutenant_governor_unknown"])
contributions_by_state_10["pct_state_house/assembly/senate_in_state"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] / (contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"])
contributions_by_state_10["pct_state_house/assembly/senate_out_of_state"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] / (contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"])
contributions_by_state_10["pct_state_house/assembly/senate_unknown"] = contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"] / (contributions_by_state_10["contributions_10_state_house/assembly/senate_in_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_out_of_state"] + contributions_by_state_10["contributions_10_state_house/assembly/senate_unknown"])

In [17]:
cols = contributions_by_state_10.columns.tolist()
cols = cols[0:5] + cols[8:9] + cols[5:8] + cols[9:10] + cols[10:16]
contributions_by_state_10 = contributions_by_state_10[cols]
contributions_by_state_10

,state,party,contributions_10_governor/lieutenant_governor_in_state,contributions_10_governor/lieutenant_governor_out_of_state,contributions_10_governor/lieutenant_governor_unknown,governor/lieutenant_governor_total,contributions_10_state_house/assembly/senate_in_state,contributions_10_state_house/assembly/senate_out_of_state,contributions_10_state_house/assembly/senate_unknown,state_house/assembly/senate_total,pct_governor/lieutenant_governor_in_state,pct_governor/lieutenant_governor_out_of_state,pct_governor/lieutenant_governor_unknown,pct_state_house/assembly/senate_in_state,pct_state_house/assembly/senate_out_of_state,pct_state_house/assembly/senate_unknown
0,AK,Democratic,"694,066.31","44,353.15",-680.00,"737,739.46","1,069,717.74","37,875.00","2,007.10","1,109,599.84",0.94,0.06,-0.00,0.96,0.03,0.00
1,AK,Republican,"2,048,733.23","116,486.30","3,016.62","2,168,236.15","1,213,813.74","36,871.50","2,693.63","1,253,378.87",0.94,0.05,0.00,0.97,0.03,0.00
2,AK,Third-Party,"1,100.00",0.00,0.00,"1,100.00","69,742.58",751.00,0.00,"70,493.58",1.00,0.00,0.00,0.99,0.01,0.00
3,AL,Democratic,"4,941,920.53","424,689.98","2,750.00","5,369,360.51","11,094,079.80","482,012.37","157,551.60","11,733,643.77",0.92,0.08,0.00,0.95,0.04,0.01
4,AL,Republican,"18,767,593.57","684,498.43","13,680.55","19,465,772.55","9,663,084.24","525,133.21","22,756.76","10,210,974.21",0.96,0.04,0.00,0.95,0.05,0.00
5,AL,Third-Party,0.00,0.00,0.00,0.00,"535,764.85","77,100.00","-12,278.82","600,586.03",nan,nan,nan,0.89,0.13,-0.02
6,AR,Democratic,"1,615,583.79","78,100.00",0.00,"1,693,683.79","2,568,106.52","274,658.93","27,156.68","2,869,922.13",0.95,0.05,0.00,0.89,0.10,0.01
7,AR,Republican,"49,448.32","4,300.00",0.00,"53,748.32","1,181,440.01","132,521.41","1,871.64","1,315,833.06",0.92,0.08,0.00,0.90,0.10,0.00
8,AR,Third-Party,0.00,0.00,0.00,0.00,"2,681.25",0.00,0.00,"2,681.25",nan,nan,nan,1.00,0.00,0.00
9,CA,Democratic,"35,235,594.21","2,800,933.56",650.00,"38,037,177.77","48,820,489.44","6,910,568.32","7,050.00","55,738,107.76",0.93,0.07,0.00,0.88,0.12,0.00


## Export the data

In [18]:
%%notify
writer = pd.ExcelWriter("data/states_analysis.xlsx")
contributions_by_state_18.to_excel(writer, "contributions_by_state_18", index=False)
contributions_by_state_14.to_excel(writer, "contributions_by_state_14", index=False)
contributions_by_state_10.to_excel(writer, "contributions_by_state_10", index=False)
writer.save()

<IPython.core.display.Javascript object>